In [47]:
params = {}

In [48]:
import time
import json
import sys

from services.dte_tools.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, get_temp_dir, get_data_dir  # noqa: E402

In [49]:
   
environment = initialise_data_task("Jupyter Sample Data Upload Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))

if not params:
    environment.log_error("No parameters given!")
    

[2025-04-22T08:38:15.152657] Jupyter Sample Data Upload Task - INFO: Jupyter Sample Data Upload Task initialised running in active mode
Jupyter Sample Data Upload Task >>>: Jupyter Sample Data Upload Task initialised running in active mode


In [50]:
assert_dte_tools_available()

resolved_parameters = get_resolved_parameters_for_connection("ANA")
resolved_server_api_parameters = get_resolved_parameters_for_connection("DUFT-SERVER-API")
tmp_dir = get_temp_dir()
environment.log_message('Data Upload Starting!')
environment.log_message("Using parameters: %s" % resolved_parameters)
environment.log_message("Using temp directory: %s" % tmp_dir)
    
print(resolved_server_api_parameters)

DTE Tools available


AttributeError: 'NoneType' object has no attribute 'directories'

In [ ]:
# Load sample dataset from SQL File

import pandas as pd
import sqlite3
import os

# Load the data
conn = sqlite3.connect(os.path.join(get_data_dir(), resolved_parameters['sqlite3file']))
data = pd.read_sql_query("SELECT * FROM dim_client LIMIT 20", conn)
conn.close()

file_to_save = os.path.join(tmp_dir, 'dim_client_sample.csv')
data.to_csv(file_to_save, index=False)

data.head()



In [ ]:
try:
    post_file_to_duft_server(
        resolved_server_api_parameters["server"], 
        resolved_server_api_parameters["username"], 
        resolved_server_api_parameters["password"], 
        file_to_save, 
        "client-list")
    
    environment.log_message('Data Upload Complete!')

except Exception as e:
    environment.log_error(f"Failed to upload file: {e}")



In [ ]:
if os.path.exists(file_to_save):
    os.remove(file_to_save)
    environment.log_message(f"Removing data file: {file_to_save}")
else:
    environment.log_message(f"File does not exist: {file_to_save}")

In [ ]:

environment.log_message('Data Upload Completed!')